<a href="https://colab.research.google.com/github/guruprasadsrinivasmurthy/Guru/blob/master/Simple_Neural_bandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import ceil
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
!pip install space_bandits


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dockers_campaigns.csv to dockers_campaigns.csv


In [ ]:
df =pd.read_csv("dockers_campaigns.csv",usecols =['Camp Rule','Send Date/Wk','Revenue'])

In [ ]:
def camp(x):
    if 'Core' in x:
        return 0 #'Core'
    elif 'Dedicated' in x:
        return 1 #'Dedicated'
    elif 'Trend' in x:
        return 2 #'Dedicated'
    elif 'NewArrivals' in x or 'NewProduct' in x:
        return 3 #'New Arrivals'
    elif 'ProductSpotlight' in x:
        return 4 #'Product Spotlight'
    elif 'Brand' in x:
        return 5 #'Brand'
    elif 'Collection' in x:
        return 6 #'Collection'
    elif 'InnovationSpotlight' in x:
        return 7 #'Innovation Spotlight'
    elif 'Tops' in x:
        return 8 # 'Tops'
    elif 'Outerwear' in x:
        return 9 #'Outerwear'
    elif 'CustomerReviews' in x:
        return  10 #'Customer Reviews'
    elif 'Replen' in x:
        return 11 #'Replen'
    else:
        return 12 #'Other'

In [ ]:
df['campaign_type'] = df['Camp Rule'].apply(lambda x: camp(x))


In [ ]:
a = set(df['campaign_type'])


In [ ]:
df['Date'] = df['Send Date/Wk'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%y'))

In [ ]:
df['dow'] = df.Date.apply(lambda x: x.weekday()).astype(float)
df['month'] = df.Date.apply(lambda x: x.month).astype(float)
df['wom'] = df.Date.apply(lambda dt: int(ceil((dt.day + dt.replace(day=1).weekday())/7.0))).astype(float)

In [ ]:
from space_bandits import NeuralBandits

In [ ]:
num_actions = 13
num_features = 3

neural_model = NeuralBandits(num_actions, num_features, initial_pulls=100, memory_size=-1, layer_sizes=[20, 12], verbose=False)
#assert neural_model.data_h.memory_size == memory_size

In [ ]:
context=pd.concat([df['dow'],df['month'],df['wom'] ],axis =1)
context= np.array(context)
Revenue = np.array(df['Revenue'])
campaign_type = np.array(df['campaign_type'])

In [ ]:
def iterate_model(model,  steps, records=None):
    """Goes through online learning simulation with model."""
    #these will track values for plotting
   
    records = dict()
    records['regret_record'] = []
    start = 0
   
    for i in range(start, start+steps):
        #get reward for 'best' choice
      
        action = model.action(context[i])
        #get reward for the action chosen by model
        reward_vec =model.expected_values(context[i])
        reward = reward_vec[action]
        #regret is the opportunity cost of not choosing the optimal promotion
        regret = ((max(reward_vec) - reward)/max(reward_vec))
        records['regret_record'].append(regret)

    numrecords=np.array(records['regret_record'])
    avg_regret=(numrecords.mean())
    return avg_regret

In [ ]:
for i in range(3):
    neural_model.fit(context,df['campaign_type'], Revenue)

regret = iterate_model(neural_model, 1440)
avg_regret.append(regret)

In [ ]:
avg_regret